In [9]:
import pandas as pd # data processing
import numpy as np # working with arrays
import matplotlib.pyplot as plt # visualization
import seaborn as sns
import random
import string
from termcolor import colored as cl # text customizationsize=len(R_7['V26'])

In [31]:
path = 'C:/Users/KABYADE/Desktop/Fraud_ML/dataset/final_fraud.xlsx'

In [32]:
# Load the credit card transaction data 
df = pd.read_excel(path, dtype = str)

In [33]:
df.CLASS.unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45',
       '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56',
       '57', '58', '59', '60', '61', '62', '63'], dtype=object)

In [45]:
cp = pd.DataFrame.copy(df)

In [63]:
import pickle

row = df.iloc[1]
row['V10'] = 643212
row['V9'] = 239806.0
row['V24'] = 777541


file = open("output/row_test.obj","wb")
pickle.dump(row, file)
file.close()

In [ ]:
data={
        "V1": "31",
        "V2": "281",
        "V3": "22",
        "V4": "010112",
        "V5": "500",
        "V6": "01/06/2009",
        "V7": "01/06/2011",
        "V8": "26/06/2009 11:16:47",
        "V9": "239806.0",
        "V10": "643212",
        "V11": None,
        "V12": "010112",
        "V13": "6011",
        "V14": None,
        "V15": None,
        "V16": "21140121124C" ,
        "V17": None,
        "V18": "6",
        "V19": "004737",
        "V20":  "0",
        "V21": "01000001",
        "V22": "01000001",
        "V23": "26/12/2009",
        "V24": "777541",
        "V25": "952",
        "V26": "384",
        "V27": "VERSUS BANK PLATEAU    REGION LAGUNECI"
}

pd.Series(data)

In [62]:
test=1

t=bool(1)
t

True

## Cleaning

In [34]:
def clean_data(df):
    #Date
    count_V3 = df.loc[df['CLASS'] != 0,'V3'].isnull().sum()
    
    df['V23'] = pd.to_datetime(df['V23'],format="%d/%m/%Y")
    df['V8'] = pd.to_datetime(df['V8'],format="%d/%m/%Y %H:%M:%S")
    
    df['V6'] = pd.to_datetime(df['V6'],format="%d/%m/%Y")
    df['V7'] = pd.to_datetime(df['V7'],format="%d/%m/%Y")
    df['V6']=df['V6'].fillna(value=df['V7'] - pd.Timedelta(days=2*365))
   
    #String
    df['V4']=df['V4'].fillna(value='010112')
    df.loc[df['V1'] == 31, 'V10'] = '00'  
    values_V3 =  np.random.choice(["002", "003"],size=count_V3).tolist()
    df['V3'] = df['V3'].apply(lambda x: values_V3.pop(0) if pd.isna(x) else x)
    df['V19'] = df['V19'].apply(lambda x: ''.join(random.choices(string.digits, k=6)) if pd.isna(x) else x)
    #Float
    df = add_amount_values(pd.DataFrame.copy(df))
    return df

def add_amount_values(df):
    fload_cols=['V9','V10','V24']
    for el in fload_cols:
        df[el] = df[el].astype(float)
        max_V10 = df[el].max()
        values = [random.randint(0, max_V10) for i in range(df[el].isnull().sum() )]
        df[el] = df[el].apply(lambda x: values.pop(0) if pd.isna(x) else x)
    return df


In [35]:
df['CLASS'] = df['CLASS'].astype(int)
df.drop(['V11','V14','V20','V5','V17','V18','V15'], axis=1, inplace=True)

In [36]:
df = clean_data(df)

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Data columns (total 21 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   V1      99999 non-null  object        
 1   V2      99999 non-null  object        
 2   V3      99999 non-null  object        
 3   V4      99999 non-null  object        
 4   V6      99999 non-null  datetime64[ns]
 5   V7      99999 non-null  datetime64[ns]
 6   V8      99999 non-null  datetime64[ns]
 7   V9      99999 non-null  float64       
 8   V10     99999 non-null  float64       
 9   V12     99999 non-null  object        
 10  V13     99999 non-null  object        
 11  V16     99999 non-null  object        
 12  V19     99999 non-null  object        
 13  V21     99999 non-null  object        
 14  V22     99999 non-null  object        
 15  V23     99999 non-null  datetime64[ns]
 16  V24     99999 non-null  float64       
 17  V25     99999 non-null  object        
 18  V26   

In [38]:
import os

try:
    os.remove("C:/Users/KABYADE/Desktop/Fraud_ML/dataset/clean_data.xlsx")
except:
    pass

df.to_excel("C:/Users/KABYADE/Desktop/Fraud_ML/dataset/clean_data.xlsx", index=False)

## Preprocessing

In [45]:
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, MinMaxScaler
import math, pickle

def update_dtype(df):
    df['CLASS'] = df['CLASS'].astype(int)
    df['V10'] = df['V10'].astype(float)
    df['V9'] = df['V9'].astype(float)
    df['V24'] = df['V24'].astype(float)
    return df

#Object preprocessing 
def to_categorical(v_df):
    cols= v_df.select_dtypes(include=['object'])
    obj_cols=cols.columns.to_list()

    for col in obj_cols:
        check=len(v_df[col].unique())
        print(str(col)+" ",check)
        le = LabelEncoder()
        v_df[col] = le.fit_transform(v_df[col])
    return v_df

shape=99999
#Object preprocessing 
def to_labels(v_df):
    
    cols = v_df.select_dtypes(include=['object'])
    obj_cols = cols.columns.to_list()
    dict_all = dict(zip([], []))
    ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value',
                                    unknown_value=-1)
    
    for col in obj_cols:
        # Fit & Transform features + save encoder as dict for future saving
        temp_keys = v_df[col].values
        temp_values = ordinal_encoder.fit_transform(v_df[col].values.reshape(-1, 1))
        temp_values = temp_values.reshape(shape).astype(int)
        v_df[col] = np.array(temp_values)
        dict_temp = dict(zip(temp_keys, temp_values))
        dict_all[col] = dict_temp
    
    for col in obj_cols:
        v_df.replace(dict_all[col], inplace=True)
    
    # saving dictionary for deploy use
    file = open( "C:/Users/KABYADE/Desktop/Fraud_ML/fraud-detection/preprocessing/output/dict_all.obj","wb")
    pickle.dump(dict_all,file)
    file.close()

    return v_df, dict_all

def process_date(df):
    #DATETIME
    list=['V6','V7','V23','V8']
    date_columns = df.filter(list)
    year_columns = pd.DataFrame()
    
    for l in list:
        date_columns[l] = pd.to_datetime(date_columns[l])
        date_columns[l+'_month'] = date_columns[l].dt.month    
        date_columns[l+'_day'] = date_columns[l].dt.day.astype(int)
        year_columns[l+'_year'] = date_columns[l].dt.year

    #print(date_columns.columns)
    date_columns.drop(list, axis=1, inplace=True)
    df.drop(list, axis=1, inplace=True)

    cos_cols = np.array(date_columns.columns.values)
    sin_cols = np.array(date_columns.columns.values)

    for i in range(len(cos_cols)):
        cos_cols[i] = 'COS(' + cos_cols[i] + ')'
        sin_cols[i] = 'SIN(' + sin_cols[i] + ')'

    # Normalize date cols
    min_max_scaler = MinMaxScaler(feature_range=(-math.pi, math.pi - (2*math.pi)/31 ))
    date_columns_scaled = min_max_scaler.fit_transform(date_columns.values)
    date_columns_cos = pd.DataFrame(date_columns_scaled, columns=cos_cols)
    date_columns_sin = pd.DataFrame(date_columns_scaled, columns=sin_cols)
    date_columns_cos = date_columns_cos.applymap(lambda x: math.cos(x))
    date_columns_sin = date_columns_sin.applymap(lambda x: math.sin(x))

    # saving scaler for deploy use
    scalerfile = "C:/Users/KABYADE/Desktop/Fraud_ML/fraud-detection/preprocessing/output/scaler.sav"
    pickle.dump(min_max_scaler, open(scalerfile, 'wb'))

    new_df = pd.concat([df, date_columns_cos, date_columns_sin, year_columns], axis=1)
    return new_df, min_max_scaler

In [44]:
clean_file="C:/Users/KABYADE/Desktop/Fraud_ML/dataset/clean_data.xlsx"
df = pd.read_excel(clean_file, dtype = str)

In [47]:
df.shape

(99999, 21)

In [48]:
df_final = update_dtype(pd.DataFrame.copy(df))
df_final, _ = process_date(pd.DataFrame.copy(df_final))

In [49]:
df_final, _ = to_labels(df_final)

In [50]:
# putting CLASS column at the end
isFraud = df_final.filter(['CLASS'],  axis=1)
df_final.drop(['CLASS'], axis=1, inplace=True)
df_final = pd.concat([df_final, isFraud], axis=1)
df_final.head()

,V1,V2,V3,V4,V9,V10,V12,V13,V16,V19,...,SIN(V7_day),SIN(V23_month),SIN(V23_day),SIN(V8_month),SIN(V8_day),V6_year,V7_year,V23_year,V8_year,CLASS
0,1,1,23,1,2000.0,104.519354,10,3715,60,56668,...,7.657137e-16,0.201299,0.937752,0.174152,-0.897805,2009,2011,2009,2009,1
1,1,1,23,1,150000.0,623643.310763,10,3660,45,41628,...,7.657137e-16,0.201299,0.937752,-0.801919,-0.998717,2009,2011,2010,2010,2
2,1,1,23,1,2000.0,269042.020672,10,3660,45,66100,...,7.657137e-16,0.201299,0.937752,0.174152,-0.968077,2009,2011,2009,2009,3
3,3,1,23,1,150000.0,606883.631345,10,5553,45,15621,...,7.657137e-16,0.201299,0.937752,-0.368810,0.848644,2009,2011,2009,2009,4
4,1,1,23,1,150000.0,150000.000000,10,2811,54,54370,...,7.657137e-16,0.201299,0.937752,-0.368810,0.848644,2009,2011,2009,2009,5


In [51]:
df_final.columns

Index(['V1', 'V2', 'V3', 'V4', 'V9', 'V10', 'V12', 'V13', 'V16', 'V19', 'V21',
       'V22', 'V24', 'V25', 'V26', 'V27', 'COS(V6_month)', 'COS(V6_day)',
       'COS(V7_month)', 'COS(V7_day)', 'COS(V23_month)', 'COS(V23_day)',
       'COS(V8_month)', 'COS(V8_day)', 'SIN(V6_month)', 'SIN(V6_day)',
       'SIN(V7_month)', 'SIN(V7_day)', 'SIN(V23_month)', 'SIN(V23_day)',
       'SIN(V8_month)', 'SIN(V8_day)', 'V6_year', 'V7_year', 'V23_year',
       'V8_year', 'CLASS'],
      dtype='object')

In [53]:
root="C:/Users/KABYADE/Desktop/Fraud_ML/dataset/"
try:
    os.remove(root + "preprocessing_data.xlsx")
except:
    pass

df_final.to_excel(root + "preprocessing_data.xlsx", index=False)